In [1]:
# Based on a paper titled "Assessing the evolution of power sector carbon intensity in the United States"
# The paper was published in IOPScience Environmental Research Letters by Greg Schivley, Inês Azevedo and Constantine Samaras
# June 4th, 2018 
# Carnegie Mellon University
# http://iopscience.iop.org/article/10.1088/1748-9326/aabe9d?utm_source=newsletter&utm_medium=email&utm_campaign=newsletter_axiosgenerate&stream=top
from collections import OrderedDict
from math import log, sqrt

import numpy as np
import pandas as pd
from six.moves import cStringIO as StringIO

from bokeh.plotting import figure, show, output_file

antibiotics = """
bacteria,                        Year_2017, Year_2008, Year_2001, gram
HI,      650,        725,            800,        above
NE,       600,         700,          725,     above
MT,                550,          700,          800,      above
AR,           500,        500,          600,        above
MI,                450,          625,            650,     above
TX,          500,        600,            620,      above
MN,                430,        690,          750,      above
IA, 430,          800,          950,    above
AK,            430,        550,            600,      above
RI,               423,      405,         430,    below
AZ,           422,          500,            550,      below
KS,           421,       800,         825,    below
OK,            420,      700,          800,    below
GA,       410,      600,           625,       below
MS,          405,      430,           575,       below
DE,          400,      700,           900,       below
"""

drug_color = OrderedDict([
    ("Year_2017",   "#0d3362"),
    ("Year_2008", "#c64737"),
    ("Year_2001",     "black"  ),
])

gram_color = {
    "above" : "#aeaeb8",
    "below" : "#e69584",
}

df = pd.read_csv(StringIO(antibiotics),
                 skiprows=1,
                 skipinitialspace=True,
                 engine='python')



In [2]:
df.head()

,bacteria,Year_2017,Year_2008,Year_2001,gram
0,HI,650,725,800,above
1,NE,600,700,725,above
2,MT,550,700,800,above
3,AR,500,500,600,above
4,MI,450,625,650,above


In [3]:
width = 800
height = 800

inner_radius = 90
outer_radius = 300 - 10

minr = sqrt(log(200 * 1E4))
maxr = sqrt(log(1000 * 1E4))
a = (outer_radius - inner_radius) / (minr - maxr)
b = inner_radius - a * maxr

def rad(mic):
    return a * np.sqrt(np.log(mic * 1E4)) + b

big_angle = 2.0 * np.pi / (len(df) + 1)
small_angle = big_angle / 7

p = figure(plot_width=width, plot_height=height, title="",
    x_axis_type=None, y_axis_type=None,
    x_range=(-420, 420), y_range=(-420, 420),
    min_border=0, outline_line_color="black",
    background_fill_color="#f0e1d2")

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

# annular wedges
angles = np.pi/2 - big_angle/2 - df.index.to_series()*big_angle
colors = [gram_color[gram] for gram in df.gram]
p.annular_wedge(
    0, 0, inner_radius, outer_radius, -big_angle+angles, angles, color=colors,
)

# small wedges
p.annular_wedge(0, 0, inner_radius, rad(df.Year_2017),
                -big_angle+angles+5*small_angle, -big_angle+angles+6*small_angle,
                color=drug_color['Year_2017'])
p.annular_wedge(0, 0, inner_radius, rad(df.Year_2008),
                -big_angle+angles+3*small_angle, -big_angle+angles+4*small_angle,
                color=drug_color['Year_2008'])
p.annular_wedge(0, 0, inner_radius, rad(df.Year_2001),
                -big_angle+angles+1*small_angle, -big_angle+angles+2*small_angle,
                color=drug_color['Year_2001'])

# circular axes and lables
labels = np.round(200*np.power(1.495348, np.arange(0, 5)))
radii = a * np.sqrt(np.log(labels * 1E4)) + b
p.circle(0, 0, radius=radii, fill_color=None, line_color="white")
p.text(0, radii[:-1], [str(r) for r in labels[:-1]],
       text_font_size="8pt", text_align="center", text_baseline="middle")

# radial axes
p.annular_wedge(0, 0, inner_radius-10, outer_radius+10,
                -big_angle+angles, -big_angle+angles, color="black")

# state labels
xr = radii[0]*np.cos(np.array(-big_angle/2 + angles))
yr = radii[0]*np.sin(np.array(-big_angle/2 + angles))
label_angle=np.array(-big_angle/2+angles)
label_angle[label_angle < -np.pi/2] += np.pi # easier to read labels on the left side
p.text(xr, yr, df.bacteria, angle=label_angle,
       text_font_size="9pt", text_align="center", text_baseline="middle")

# Legend
p.circle([-40, -40], [-370, -390], color=list(gram_color.values()), radius=5)
p.text([-30, -30], [-370, -390], text=["In 2017: 425 g CO2/kWh-" + gr for gr in gram_color.keys()],
       text_font_size="7pt", text_align="left", text_baseline="middle")

p.rect([-40, -40, -40], [18, 0, -18], width=30, height=13,
       color=list(drug_color.values()))
p.text([-15, -15, -15], [18, 0, -18], text=list(drug_color),
       text_font_size="9pt", text_align="left", text_baseline="middle")

output_file("State_CO2_Levels.html", title="State CO2 Levels")

show(p)